##**Data Preprocessing , Annotation and Feature extraction**

In [ ]:
import cv2
import dlib
import numpy as np
import json
import os
import shutil
from sklearn.model_selection import train_test_split
video_path="/content/drive/MyDrive/Experimenter_9110002_53.mp4"
# Load the pre-trained face detector and facial landmarks predictor from dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/content/drive/MyDrive/shape_predictor_68_face_landmarks (1).dat")

# Define EAR and MAR thresholds
EAR_THRESHOLD = 0.2
MAR_THRESHOLD = 0.4

# Function to calculate eye aspect ratio (EAR)
def eye_aspect_ratio(eye):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Function to calculate mouth aspect ratio (MAR)
def mouth_aspect_ratio(mouth):
    A = np.linalg.norm(mouth[14] - mouth[18])
    B = np.linalg.norm(mouth[12] - mouth[16])
    mar = A / B
    return mar

# Function to detect faces, annotate frames, and save annotated frames with labels
def annotate_frames_with_labels(video_path, frames_dir, annotations_file):
    cap = cv2.VideoCapture(video_path)
    annotations = []
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray)

        for face in faces:
            x, y, w, h = face.left(), face.top(), face.width(), face.height()
            landmarks = predictor(gray, face)
            landmarks = np.array([[p.x, p.y] for p in landmarks.parts()])
            left_eye = landmarks[36:42]
            right_eye = landmarks[42:48]
            mouth = landmarks[48:68]
            ear_left = eye_aspect_ratio(left_eye)
            ear_right = eye_aspect_ratio(right_eye)
            mar = mouth_aspect_ratio(mouth)

            label = "Attentive"
            if ear_left < EAR_THRESHOLD or ear_right < EAR_THRESHOLD or mar > MAR_THRESHOLD:
                label = "Drowsy"

            annotations.append({
                "frame_name": f"frame_{frame_count}.jpg",
                "face_bbox": [x, y, w, h],
                "ear_left": ear_left,
                "ear_right": ear_right,
                "mar": mar,
                "label": label
            })

            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, f"Label: {label}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            frame_path = os.path.join(frames_dir, f"frame_{frame_count}.jpg")
            cv2.imwrite(frame_path, frame)

            frame_count += 1

    cap.release()

    with open(annotations_file, "w") as f:
        json.dump(annotations, f)

# Function to visualize annotated frames
def visualize_annotated_frames(frames_dir, annotations_file, num_frames=5):
    with open(annotations_file, "r") as f:
        annotations = json.load(f)

    for i, annotation in enumerate(annotations[:num_frames]):
        frame_path = os.path.join(frames_dir, annotation["frame_name"])
        frame = cv2.imread(frame_path)

        x, y, w, h = map(int, annotation["face_bbox"])
        label = annotation["label"]

        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, f"Label: {label}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        cv2.imshow(f"Annotated Frame {i+1}", frame)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

# Define paths
video_path = "/content/drive/MyDrive/Experimenter_9110002_53.mp4"  # Change this to the path of your video file
save_path = "/content/drive/MyDrive/preprocessed_video"

# Create save directory if not exists
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Call annotation function
annotate_frames_with_labels(video_path, save_path, os.path.join(save_path, "annotations.json"))



###**Data Splitting**

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split
import json

# Define paths
parent_folder = "/content/drive/MyDrive/preprocessed_video"  # Update with the path to your parent folder containing annotated frames and JSON file
train_folder = os.path.join(parent_folder, "train")
test_folder = os.path.join(parent_folder, "test")
annotations_file = os.path.join(parent_folder, "/content/drive/MyDrive/preprocessed_video/annotations.json")

# Load annotations
with open(annotations_file, "r") as f:
    annotations = json.load(f)

# Extract frame names and labels
frame_names = [annotation["frame_name"] for annotation in annotations]
labels = [annotation["label"] for annotation in annotations]

# Split data into training and testing sets (80-20 ratio)
train_frames, test_frames, train_annotations, test_annotations = train_test_split(frame_names, annotations, test_size=0.2, random_state=42)

# Create train and test folders if they don't exist
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Write training annotations to a new JSON file
train_annotations_file = os.path.join(train_folder, "annotations.json")
with open(train_annotations_file, "w") as f:
    json.dump(train_annotations, f)

# Write testing annotations to a new JSON file
test_annotations_file = os.path.join(test_folder, "annotations.json")
with open(test_annotations_file, "w") as f:
    json.dump(test_annotations, f)

# Move training frames to the train folder
for frame in train_frames:
    src = os.path.join(parent_folder, frame)
    if os.path.exists(src):
        shutil.move(src, os.path.join(train_folder, frame))
    else:
        print(f"File not found: {src}")

# Move testing frames to the test folder
for frame in test_frames:
    src = os.path.join(parent_folder, frame)
    if os.path.exists(src):
        shutil.move(src, os.path.join(test_folder, frame))
    else:
        print(f"File not found: {src}")

print("Data split and files moved successfully.")


##**Model Training and Evaluation**

In [ ]:
# Importing necessary libraries
import os
import json
import numpy as np
!pip install --upgrade scikit-learn
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

In [7]:

# Define paths
train_folder = "/content/drive/MyDrive/preprocessed_video/train"
test_folder = "/content/drive/MyDrive/preprocessed_video/test"

# Load annotations
train_annotations_file = os.path.join(train_folder, "annotations.json")
test_annotations_file = os.path.join(test_folder, "annotations.json")

with open(train_annotations_file, "r") as f:
    train_annotations = json.load(f)

with open(test_annotations_file, "r") as f:
    test_annotations = json.load(f)

# Extract features and labels for drowsiness
X_train_drowsy = [[annotation["ear_left"], annotation["ear_right"], annotation["mar"]] for annotation in train_annotations]
y_train_drowsy = [1 if annotation["label"] == "Drowsy" else 0 for annotation in train_annotations]

X_test_drowsy = [[annotation["ear_left"], annotation["ear_right"], annotation["mar"]] for annotation in test_annotations]
y_test_drowsy = [1 if annotation["label"] == "Drowsy" else 0 for annotation in test_annotations]

# Extract features and labels for attentiveness
X_train_attentive = [[annotation["ear_left"], annotation["ear_right"], annotation["mar"]] for annotation in train_annotations]
y_train_attentive = [1 if annotation["label"] == "Attentive" else 0 for annotation in train_annotations]

X_test_attentive = [[annotation["ear_left"], annotation["ear_right"], annotation["mar"]] for annotation in test_annotations]
y_test_attentive = [1 if annotation["label"] == "Attentive" else 0 for annotation in test_annotations]

# Train the SVM model for drowsiness
model_drowsy = SVC(probability=True)
model_drowsy.fit(X_train_drowsy, y_train_drowsy)

# Train the SVM model for attentiveness
model_attentive = SVC(probability=True)
model_attentive.fit(X_train_attentive, y_train_attentive)

# Evaluate the model for drowsiness
y_pred_drowsy = model_drowsy.predict(X_test_drowsy)
report_drowsy = classification_report(y_test_drowsy, y_pred_drowsy)
conf_matrix_drowsy = confusion_matrix(y_test_drowsy, y_pred_drowsy)

# Evaluate the model for attentiveness
y_pred_attentive = model_attentive.predict(X_test_attentive)
report_attentive = classification_report(y_test_attentive, y_pred_attentive)
conf_matrix_attentive = confusion_matrix(y_test_attentive, y_pred_attentive)
# Calculate number of labels
num_train_drowsy = sum(y_train_drowsy)
num_test_drowsy = sum(y_test_drowsy)
num_train_attentive = sum(y_train_attentive)
num_test_attentive = sum(y_test_attentive)

# Print number of labels
print(f"Number of Drowsy Labels in Training Data: {num_train_drowsy}")
print(f"Number of Drowsy Labels in Testing Data: {num_test_drowsy}")
print()
print(f"Number of Attentive Labels in Training Data: {num_train_attentive}")
print(f"Number of Attentive Labels in Testing Data: {num_test_attentive}")
print()

# Evaluate the model for drowsiness
prob_drowsy = model_drowsy.predict_proba(X_test_drowsy)[:, 1]  # Probability estimates for the positive class (Drowsy)
drowsiness_percentage = np.mean(prob_drowsy) * 100

# Evaluate the model for attentiveness
prob_attentive = model_attentive.predict_proba(X_test_attentive)[:, 1]  # Probability estimates for the positive class (Attentive)
attentiveness_percentage = np.mean(prob_attentive) * 100

# Define qualification levels
drowsiness_qualification = "Very Drowsy" if drowsiness_percentage > 60 else "Drowsy" if drowsiness_percentage > 30 else "Slightly Drowsy"
attentiveness_qualification = "Very Attentive" if attentiveness_percentage > 60 else "Attentive" if attentiveness_percentage > 30 else "Slightly Attentive"

# Print qualifications along with percentages
print(f"Drowsiness Level: {drowsiness_qualification}, Percentage: {drowsiness_percentage:.2f}%")
print(f"Attentiveness Level: {attentiveness_qualification}, Percentage: {attentiveness_percentage:.2f}%")
# Print classification report for drowsiness
print()
print("Classification Report for Drowsiness:")
print(report_drowsy)
print("Confusion Matrix for Drowsiness:")
print(conf_matrix_drowsy)

# Print classification report for attentiveness
print()
print("Classification Report for Attentiveness:")
print(report_attentive)
print("Confusion Matrix for Attentiveness:")
print(conf_matrix_attentive)



Number of Drowsy Labels in Training Data: 1750
Number of Drowsy Labels in Testing Data: 442

Number of Attentive Labels in Training Data: 14200
Number of Attentive Labels in Testing Data: 3546

Drowsiness Level: Slightly Drowsy, Percentage: 11.18%
Attentiveness Level: Very Attentive, Percentage: 88.82%

Classification Report for Drowsiness:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3546
           1       0.97      0.97      0.97       442

    accuracy                           0.99      3988
   macro avg       0.98      0.98      0.98      3988
weighted avg       0.99      0.99      0.99      3988

Confusion Matrix for Drowsiness:
[[3532   14]
 [  13  429]]

Classification Report for Attentiveness:
              precision    recall  f1-score   support

           0       0.97      0.97      0.97       442
           1       1.00      1.00      1.00      3546

    accuracy                           0.99      3988
   macro a